In [185]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from sklearn.preprocessing import scale
import urllib, json

# 행정동 데이터 전처리

In [186]:
# 유동인구 데이터 불러오기

f = pd.read_csv("fpopl.csv")
df = pd.DataFrame(f)
df.shape

(47546443, 6)

In [187]:
# 행정동 데이터 불러오기

f2 = pd.read_csv("adstrd_master.csv")
df2 = pd.DataFrame(f2)

df2.shape

(3493, 4)

In [188]:
# df와 df2 합치기 (유동인구['adstrd_code'] = 행정동['adstrd_code'])
#                          * adstrd_code = 행정동코드
# 유동인구 데이터와 행정동 이름 합침
# SQL의 LEFT JOIN

move_people = pd.merge(df,df2,on="adstrd_code",how="left")
move_people.head()

,base_ymd,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,popltn_cascnt,adstrd_nm,brtc_nm,signgu_nm
0,20200101,0,M,age_00,11650560,15,반포1동,서울특별시,서초구
1,20200101,0,M,age_00,11590620,6,사당1동,서울특별시,동작구
2,20200101,0,M,age_00,11560710,4,대림2동,서울특별시,영등포구
3,20200101,0,M,age_00,11470680,12,신정7동,서울특별시,양천구
4,20200101,0,M,age_00,11350665,6,상계3.4동,서울특별시,노원구


In [189]:
del move_people['brtc_nm']

# brtc_nm은 시도 명이다.
# 현 DATASET은 서울시 내 행정구역 데이터이기 때문에, brtc_nm 칼럼 삭제

In [190]:
move_people = move_people.rename({'base_ymd':'년월일','tmzon_se_code':'시간대','sexdstn_se_code':'성별','agrde_se_code':'나이대','adstrd_code':'행정동코드','popltn_cascnt':'유동인구','adstrd_nm':'행정동','signgu_nm':'행정구'}, axis ='columns')

grouped = move_people.groupby(by = ['년월일','행정구']).sum()['유동인구'].reset_index()

grouped

,년월일,행정구,유동인구
0,20200101,강남구,3028808
1,20200101,강동구,2094980
2,20200101,강북구,945218
3,20200101,강서구,2191177
4,20200101,관악구,1694584
...,...,...,...
4195,20200616,용산구,2363994
4196,20200616,은평구,1610731
4197,20200616,종로구,1992243
4198,20200616,중구,2253617


In [191]:
grouped['주차'] = grouped['년월일']%20200000
grouped

grouped['월'] = round(grouped['주차']/100)
grouped['일'] = grouped['주차']%100 + 30* (grouped['월']-1) +2

grouped['주차'] = round(grouped['일']/7+1)
del grouped['월']

grouped = grouped.astype({'주차': 'int'})
del grouped['일']
grouped

,년월일,행정구,유동인구,주차
0,20200101,강남구,3028808,1
1,20200101,강동구,2094980,1
2,20200101,강북구,945218,1
3,20200101,강서구,2191177,1
4,20200101,관악구,1694584,1
...,...,...,...,...
4195,20200616,용산구,2363994,25
4196,20200616,은평구,1610731,25
4197,20200616,종로구,1992243,25
4198,20200616,중구,2253617,25


In [198]:
grouped['유동인구_표준화'] = scale(grouped['유동인구'])

grouped_avg = grouped.mean()
grouped['유동인구_평균대비_증감률'] = (grouped['유동인구']-grouped_avg[1])/grouped_avg[1]*100
grouped['유동인구_코로나전평균'] = round(grouped['유동인구']-grouped.iloc[0:35]['유동인구'].mean())
grouped

,년월일,행정구,유동인구,주차,유동인구_표준화,유동인구_평균대비_증감률,유동인구_코로나전평균
0,20200101,강남구,3028808,1,1.035484,43.302717,1132153.0
1,20200101,강동구,2094980,1,-0.021036,-0.879710,198325.0
2,20200101,강북구,945218,1,-1.321861,-55.278675,-951437.0
3,20200101,강서구,2191177,1,0.087800,3.671682,294522.0
4,20200101,관악구,1694584,1,-0.474039,-19.823742,-202071.0
...,...,...,...,...,...,...,...
4195,20200616,용산구,2363994,25,0.283322,11.848214,467339.0
4196,20200616,은평구,1610731,25,-0.568909,-23.791099,-285924.0
4197,20200616,종로구,1992243,25,-0.137271,-5.740530,95588.0
4198,20200616,중구,2253617,25,0.158443,6.625920,356962.0


# 지도 그리기

## 서울 자치구 경계 json파일 다운로드

In [210]:
url = 'https://raw.github.com/PinkWink/DataScience/master/data/02.%20skorea_municipalities_geo_simple.json'
with urllib.request.urlopen(url) as fp:
  json_seoul = json.loads(fp.read())

In [211]:
#for idx, seoul_dict in enumerate(json_seoul['features']):
#  gu_name = seoul_dict['id']
#  value = temp.loc[temp.행정구==gu_name, '유동인구'].iloc[0]
#  txt = f'{gu_name}<br>{value}'
#  json_seoul['features'][idx]['properties']['tooltip1'] = txt

In [212]:
#m = folium.Map(location=[37.55, 127], zoom_start=11, tiles='Stamen Terrain')


In [213]:
def print_map(args):
#    print(args)

    clear_output(wait = True)
    display(button1)
    display(button2)
    m = folium.Map(location=[37.55, 127], zoom_start=11, tiles='Stamen Terrain')
    
    temp = grouped[grouped.주차==args]
    for idx, seoul_dict in enumerate(json_seoul['features']):
        gu_name = seoul_dict['id']
        value = temp.loc[temp.행정구==gu_name, '유동인구_코로나전평균'].iloc[0]
        corr = False;
        txt = f'{gu_name}<br>{value:.2f}'
        json_seoul['features'][idx]['properties']['tooltip1'] = txt
        
    choropleth = folium.Choropleth(
        geo_data=json_seoul, 
        data=temp,
        columns=['행정구','유동인구_코로나전평균'],
        fill_color = 'PuBuGn',
        key_on = 'feature.id',
        fill_opacity=0.9,
        line_opacity=1,
        legend_name='Foot Traffic'
    ).add_to(m)

    choropleth.geojson.add_child( folium.features.GeoJsonTooltip(['tooltip1'], labels=False))

    folium.LayerControl().add_to(m)

    title_html = '<h3 align="center" style="font-size:20px"><b>Foot Traffic in Seoul on '+str(args)+' week</b></h3>'
    m.get_root().html.add_child(folium.Element(title_html))

    display(m)
    
date = 1

button1 = widgets.Button(description='다음 주 보기')
button2 = widgets.Button(description='이전 주 보기')
def clicked_next(arg):
    global date
    if date <25:
        date = date + 1
        print_map(date)


def clicked_prev(arg):
    global date
    if date > 1: 
        date = date - 1
        print_map(date)
    

button1.on_click(clicked_next)
button2.on_click(clicked_prev)
display(button2)
display(button1)
print_map(date)

Button(description='다음 주 보기', style=ButtonStyle())

Button(description='이전 주 보기', style=ButtonStyle())